In [2]:
import pandas as pd
from openpyxl import load_workbook
import numpy as np
import datetime

Update list of polymers based on raw data collected

In [ ]:
polymers = pd.read_excel('Data/Melt Viscosity Data.xlsx', ["Homopolymer", "Copolymer", "Polymer Blend"])
df_master = pd.read_pickle("database_2021_11_30.pkl")
columns = ['rid', 'smiles', 'fingerprints']
df_master = df_master[columns]

In [3]:
polymers = pd.read_excel('Data/Validation Data.xlsx', ["Homopolymer", "Copolymer", "Polymer Blend"])
df_master = pd.read_pickle("Data/database_2021_11_30.pkl")
columns = ['rid', 'smiles', 'fingerprints']
df_master = df_master[columns]

In [4]:
SMILES_df = pd.DataFrame(columns = ['Polymer', 'SMILES', 'PID'])
SMILES_df = pd.concat([SMILES_df, polymers["Homopolymer"][['Polymer','SMILES', 'PID']]], ignore_index = True)
SMILES_df = pd.concat([SMILES_df, polymers["Copolymer"].rename(columns = {'Polymer 1': 'Polymer', 'SMILES 1': 'SMILES', 'PID 1':'PID'}).loc[:,['Polymer','SMILES', 'PID']]], ignore_index = True)
SMILES_df = pd.concat([SMILES_df, polymers["Copolymer"].rename(columns = {'Polymer 2': 'Polymer', 'SMILES 2': 'SMILES', 'PID 2':'PID'}).loc[:,['Polymer','SMILES', 'PID']]], ignore_index = True)
SMILES_df = pd.concat([SMILES_df, polymers["Polymer Blend"].rename(columns = {'Polymer 1': 'Polymer', 'SMILES 1': 'SMILES', 'PID 1':'PID'}).loc[:,['Polymer','SMILES', 'PID']]], ignore_index = True)
SMILES_df = pd.concat([SMILES_df, polymers["Polymer Blend"].rename(columns = {'Polymer 2': 'Polymer', 'SMILES 2': 'SMILES', 'PID 2':'PID'}).loc[:,['Polymer','SMILES', 'PID']]], ignore_index = True)
SMILES_df = SMILES_df.drop_duplicates('Polymer').reset_index().drop(columns=['index'])
SMILES_df['SMILES'] = SMILES_df['SMILES'].astype(str)

In [5]:
SMILES_df.to_excel('Polymer-SMILES2.xlsx', sheet_name ='Polymers', index = False)

Update/read in custom fp file

In [57]:
new_fp = pd.read_csv('SMILES-2022-01-10-FP.csv')
custom_fp = pd.read_excel('Data/custom_fp.xlsx')
custom_fp.append(new_fp).to_excel('Data/custom_fp.xlsx', index = False)

Read in updated list with SMILES to get polymer fingerprints

In [6]:
custom_fp = pd.read_excel('Data/custom_fp.xlsx')
file_name = 'Polymer-SMILES2.xlsx'
SMILES_df = pd.read_excel(file_name)
IDcon = pd.read_excel('Data/polymer_master_data_v20200625_1.xlsx')
SMILES_df['SMILES'] = SMILES_df['SMILES'].astype(str)

In [20]:
df_master

,rid,smiles,fingerprints
0,R1000001,[*]C[*],"{'afp_C4_C4_C4': 0.3333333333333333, 'afp_C4_C..."
1,R1000001,[*]C[*],"{'afp_C4_C4_C4': 0.3333333333333333, 'afp_C4_C..."
2,R1000001,[*]C[*],"{'afp_C4_C4_C4': 0.3333333333333333, 'afp_C4_C..."
3,R1000001,[*]C[*],"{'afp_C4_C4_C4': 0.3333333333333333, 'afp_C4_C..."
4,R1000001,[*]C[*],"{'afp_C4_C4_C4': 0.3333333333333333, 'afp_C4_C..."
...,...,...,...
29807,R1014114,S=C([*])CCCC(C(C)(C)C)S[*],NaN
29808,R1014115,S=C([*])CCCC(C(C)C(C)C)S[*],NaN
29809,R1014116,S=C([*])CCCC(CC(C)(C)C)S[*],NaN
29810,R1014117,S=C([*])CCCC(C(C)C(C)(C)C)S[*],NaN


In [7]:
cust_fp_col = [col for col in custom_fp.columns if 'fp' in col]
fp_df = pd.DataFrame()
need_fp = pd.DataFrame()
need_fp.index.names = ['INDEX']
for i in SMILES_df.index:
    if '*' in SMILES_df.loc[i, 'SMILES'] or 'C' in SMILES_df.loc[i, 'SMILES']:
        if len(df_master.loc[df_master['smiles'] == SMILES_df.loc[i, 'SMILES']].reset_index(drop = True)) == 0:
            if len(custom_fp.loc[custom_fp['SMILES'] == SMILES_df.loc[i, 'SMILES']].reset_index(drop = True)) == 0:
                print('Need FP for ' + SMILES_df.loc[i, 'SMILES'])
                need_fp.loc[len(need_fp), 'SMILES'] = SMILES_df.loc[i, 'SMILES']
                continue
            else:
                fp = pd.DataFrame(custom_fp.loc[custom_fp['SMILES'] == SMILES_df.loc[i, 'SMILES']].reset_index(drop = True).loc[0, cust_fp_col])
                print(fp)
        else:            
            fp = pd.DataFrame(df_master.loc[df_master['smiles'] == SMILES_df.loc[i, 'SMILES']].reset_index(drop = True).loc[0, 'fingerprints'], index = [i])
        fp.loc[i, 'SMILES'] = SMILES_df.loc[i, 'SMILES']
        fp.loc[i, 'Polymer'] = SMILES_df.loc[i, 'Polymer']
        fp_df = fp_df.append(fp, ignore_index = True)
    elif pd.isna(SMILES_df.loc[i, 'PID']) == False:
        if len( IDcon.loc[IDcon['PID'] == SMILES_df.loc[i, 'PID'], 'RID'].values) == 0:
            print('Need SMILES for ' + str(SMILES_df.loc[i, 'PID']))
            continue
        RID = IDcon.loc[IDcon['PID'] == SMILES_df.loc[i, 'PID'], 'RID'].values[0]
        fp = pd.DataFrame(df_master.loc[df_master['rid'] == RID].reset_index(drop = True).loc[0, 'fingerprints'], index = [i])
        fp.loc[i, 'SMILES'] = df_master.loc[df_master['rid'] == RID].reset_index(drop = True).loc[0, 'smiles']
        fp.loc[i, 'Polymer'] = SMILES_df.loc[i, 'Polymer']
        fp_df = fp_df.append(fp, ignore_index = True)

Need SMILES for P342449


/tmp/ipykernel_170021/2379119993.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fp_df = fp_df.append(fp, ignore_index = True)
/tmp/ipykernel_170021/2379119993.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fp_df = fp_df.append(fp, ignore_index = True)
/tmp/ipykernel_170021/2379119993.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fp_df = fp_df.append(fp, ignore_index = True)
/tmp/ipykernel_170021/2379119993.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fp_df = fp_df.append(fp, ignore_index = True)
/tmp/ipykernel_170021/2379119993.py:28: FutureWarning: The frame.append method is deprecated and will be removed fro

In [9]:
cols = list(fp_df)
cols.insert(0, cols.pop(cols.index('SMILES')))
cols.insert(0, cols.pop(cols.index('Polymer')))
fp_df = fp_df[cols].drop_duplicates('SMILES').fillna(0)

In [10]:
fp_df

,Polymer,SMILES,afp_C4_C4_C4,afp_C4_C4_O2,afp_C4_C4_F1,afp_C4_O2_C4,afp_F1_C4_O2,afp_F1_C4_F1,bfp_299,bfp_304,...,bfp_114,bfp_145,bfp_222,bfp_311,bfp_343,mfp_MQNs17,mfp_MQNs36,mfp_NumAromaticRings,bfp_218,bfp_376
0,"poly[oxy(1,1,2,2,3,3-hexafluoropropane-1,3-diyl)]",[*]OC(F)(F)C(F)(F)C([*])(F)F,0.100000,0.200000,0.8,0.1,0.4,0.3,0.9,0.300000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"poly[(decane-1,10-diamine)-alt-(decanedioic ac...",[*]CCCCCCCCCCNC(=O)CCCCCCCCC(=O)N[*],0.225806,0.000000,0.0,0.0,0.0,0.0,0.0,0.774194,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,polyethene,[*]C[*],0.333333,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"poly(5-ethylidenebicyclo[2.2.1]heptane-2,3-diyl)",[*]C1C2CC(=CC)C(C2)C1[*],0.523810,0.000000,0.0,0.0,0.0,0.0,0.0,2.142857,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,poly(methyl methacrylate),[*]CC([*])(C)C(=O)OC,0.266667,0.000000,0.0,0.0,0.0,0.0,0.0,1.400000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,poly(lactic acid),[*]OC(C)C([*])=O,0.000000,0.111111,0.0,0.0,0.0,0.0,0.0,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,poly(p-hydroxystyrene),[*]CC([*])c1ccc(O)cc1,0.117647,0.000000,0.0,0.0,0.0,0.0,0.0,0.705882,...,0.647059,0.588235,0.352941,0.117647,0.705882,0.176471,0.058824,0.058824,0.000000,0.000000
7,poly(epsilon-caprolactone),[*]CCCCCC(=O)O[*],0.166667,0.055556,0.0,0.0,0.0,0.0,0.0,0.666667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.388889,0.444444
8,poly(vinyl acetate),[*]CC([*])OC(C)=O,0.166667,0.166667,0.0,0.0,0.0,0.0,0.0,0.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
if file_name == 'Data/EntanglementMW.xlsx':
    ent_fp = fp_df.join(SMILES_df['Me'])
    ent_fp.to_excel('Data/EntanglementMW_fp.xlsx', sheet_name ='Polymers')
if file_name != 'Polymer-SMILES.xlsx':
    old_fp = pd.read_excel('Polymer-SMILES.xlsx')
    fp_df = pd.concat([fp_df, old_fp])

In [11]:
fp_df.to_excel('Polymer-SMILES-FP2.xlsx')

In [11]:
need_fp

""
INDEX


In [12]:
need_fp.to_excel('SMILES-' + str(datetime.date.today()) + '.xlsx')